In [253]:
import time

In [254]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [255]:
wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"
addr = "https://entertain.naver.com/ranking/comment/list?oid=112&aid=0003233365"

In [256]:
driver = webdriver.Chrome(wd)
driver.get(addr)

In [257]:
pages = 0

In [258]:
try:
    while True:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
        time.sleep(1.5)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
    
except Exception as e: # 다른 예외 발생시 확인
    print(e)

0123456789101112131415161718192021Message: 



In [259]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [260]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]


In [261]:
comments[:3]

['머리잘랏구나 ㅋㅋ 잘어울림', '단발머리도 너무 잘어울린당', '화사는 목이랑 어깨라인이 진짜 예술임 ....ㅜㅜ 이쁘다']

In [262]:
p_comment_list = []

In [263]:
for comment in comments:
    p_comment_list.append(comment)

In [264]:
file = open('p_comment36.txt','w',encoding = 'utf-8')
for comment in p_comment_list:
    file.write(comment + '\n')
file.close()